# Calculating the Bhattacharryya bound on compare it with ML decoding

This code (MATLAB) is provided as supplementary material of the lecture Channel Coding - Graph-based Codes.

This code illustrates
* Calculating the Bhattacharryya bound on compare it with ML decoding for a specific code

In [1]:
% specify code by its generator matrix
G = [1 0 0 0 1 1 0;
     0 1 0 0 1 0 1;
     0 0 1 0 0 1 1];

In [2]:
% get parameters k and n of the code
k = size(G,1);
n = size(G,2);

% generate table of all codewords
% GF(2) operations are obtained by calculating modulo-2
u = de2bi(0:(2^k-1),k,'left-msb');  % all possible input patterns
code = mod(u * G, 2);  % generate codewords

In [3]:
% compute weights
w = sum(code,2);
dmin = min(w(2:end));  % exclude all-zero codeword

% weight enumerator polynomial
A = [fliplr(histc(w, 1:n)'), 0];

In [7]:
poly2sym(A)

poly2sym requires Symbolic Math Toolbox.